In [0]:
fileroot = "clinicaltrial_2021"
import os
os.environ['fileroot'] = fileroot

In [0]:
Clinicaltrial_file = ("/FileStore/tables/"+fileroot+".csv")
Mesh = ('dbfs:/FileStore/tables/mesh.csv')
Pharma = ('dbfs:/FileStore/tables/pharma.csv')

In [0]:
dbutils.fs.head('/FileStore/tables/' + fileroot + '.csv')

[Truncated to first 65536 bytes]
Out[3]: "Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions\r\nNCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||\r\nNCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|\r\nNCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|\r\nNCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|\r\nNCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|\r\nNCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate\r\nNCT02752438|Ankara University|Unkn

In [0]:
for line in dbutils.fs.head('/FileStore/tables/' + fileroot + '.csv').splitlines():
    print (line)

[Truncated to first 65536 bytes]
Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions
NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||
NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|
NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|
NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|
NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|
NCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate
NCT02752438|Ankara University|Unknown status|May 2016|Jul 2017|O

In [0]:
Clinicaltrial_2021 = sc.textFile(Clinicaltrial_file) 
header = Clinicaltrial_2021.first() 
Clinicaltrial_2021 = Clinicaltrial_2021.filter(lambda row:  row != header) 

In [0]:
No_of_Studies_RDD = Clinicaltrial_2021.distinct().count()
print(No_of_Studies_RDD)

387261


In [0]:
Type_header = Clinicaltrial_2021.first()
Type_Frequencies_RDD = Clinicaltrial_2021.filter(lambda row:  row != Type_header) 
Type_Frequencies_RDD1 = Clinicaltrial_2021.map(lambda p: p.split('|')) 
Type_count = Type_Frequencies_RDD1.map(lambda p: (p[5]))\
.map(lambda q: (q,1))\
.reduceByKey(lambda p1, p2: p1+p2)\
.map(lambda x: (x[1],x[0]))\
.sortByKey(ascending=False)\
.map(lambda x: (x[1], x[0]))

In [0]:
Type_count.collect()

Out[8]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
Conditions_header = Clinicaltrial_2021.first() 
Conditions_Frequencies_RDD = Clinicaltrial_2021.filter(lambda row:  row != header)
Conditions_Frequencies_RDD1 = Conditions_Frequencies_RDD.map(lambda c: c.split('|'))\
.flatMap(lambda a: a[7].split(','))\
.map(lambda b: [b, 1])\
.reduceByKey(lambda c1, c2: c1+c2)\
.map(lambda d: (d[1],d[0]))\
.sortByKey(ascending=False)\
.map(lambda e: (e[1],e[0]))\
.filter(lambda f: (f[0] != ''))
Conditions_Frequencies_RDD1.take(5)

Out[9]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
Mesh_RDD = sc.textFile(Mesh)
Mesh_header = Mesh_RDD.first() 
Mesh_RDD = Mesh_RDD.filter(lambda row:  row != Mesh_header) 
Splitted_indiced_Mesh = Mesh_RDD.map(lambda x:(x.split(",")[0],x.split(",")[1].split(".")[0]))
Splitted_indiced_Mesh.take(5)

Out[10]: [('Calcimycin', 'D03'),
 ('A-23187', 'D03'),
 ('Temefos', 'D02'),
 ('Temefos', 'D02'),
 ('Temefos', 'D02')]

In [0]:
Conditions_header = Clinicaltrial_2021.first() 
Conditions_Column_RDD = Clinicaltrial_2021.filter(lambda row:  row != Conditions_header)
Conditions_Column_RDD1 = Conditions_Column_RDD.map(lambda c: c.split('|'))\
.flatMap(lambda g: g[7].split(','))\
.filter(lambda h: h != '')\
.map(lambda i: [i, 1])
Conditions_Column_RDD1.take(5)

Out[11]: [['Autistic Disorder', 1],
 ['Autism Spectrum Disorder', 1],
 ['Diabetes Mellitus', 1],
 ['Tuberculosis', 1],
 ['Lung Diseases', 1]]

In [0]:
Column_Join = Conditions_Column_RDD1.join(Splitted_indiced_Mesh)
Frequent_Roots = Column_Join.map(lambda j: (j[1][1], 1))\
.reduceByKey(lambda k1, k2: k1+ k2)\
.sortBy(lambda l: -l[1])
Frequent_Roots.take(5)

Out[12]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
Pharma_RDD = sc.textFile(Pharma)
Pharma_header = Pharma_RDD.first()
Pharma_RDD = Pharma_RDD.filter(lambda row:  row != Pharma_header) 
Parent_Company_Column = Pharma_RDD.map(lambda m: m.split(','))\
.map(lambda n: (n[1].replace('"', '')))\
.map(lambda o: (o,1))
Parent_Company_Column.take(10)

Out[13]: [('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 (' Inc.', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 (' Inc.', 1),
 ('Johnson & Johnson', 1),
 (' a division of Ethicon Inc.', 1),
 ('Abbott Laboratories', 1)]

In [0]:
Sponsor_header = Clinicaltrial_2021.first()
Sponsor_Column_RDD = Clinicaltrial_2021.filter(lambda row:  row != Sponsor_header)
Sponsor_Column_RDD1 = Sponsor_Column_RDD.map(lambda clicnical: clicnical.split('|'))\
.map(lambda p: p[1])\
.map(lambda q: (q, 1))
Sponsor_Column_RDD1.take(10)

Out[14]: [('Duke University', 1),
 ('Universidade Federal do Rio de Janeiro', 1),
 ('Istanbul Medeniyet University', 1),
 ('University of Roma La Sapienza', 1),
 ('Consorzio Futuro in Ricerca', 1),
 ('Ankara University', 1),
 ('Ruijin Hospital', 1),
 ('Washington University School of Medicine', 1),
 ('Orphazyme', 1),
 ('Novo Nordisk A/S', 1)]

In [0]:
Column_Join = Sponsor_Column_RDD1.leftOuterJoin(Parent_Company_Column)
Not_Pharmaceutical_Companies = Column_Join.filter(lambda r : r[1][1] == None)\
.map(lambda s: (s[0], 1))\
.reduceByKey(lambda t1, t2: t1+ t2)\
.sortBy(lambda u: -u[1])
Not_Pharmaceutical_Companies.take(10)

Out[15]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
Completed_header = Clinicaltrial_2021.first()
Completed_Studies_RDD = Clinicaltrial_2021.filter(lambda row:  row != Completed_header)
Completed_Studies_RDD = Completed_Studies_RDD .map(lambda clicnical: clicnical.split('|'))
Completed_studies1 = Completed_Studies_RDD.filter(lambda v: v[2] == 'Completed')\
.map(lambda w: w[4].split(' '))\
.filter(lambda x: x[0] != '')\
.filter(lambda y : y[1] == '2021')\
.map(lambda z: (z[0],1))\
.reduceByKey(lambda a1,a2: a1+ a2)\
.sortBy(lambda b: b[0])
Completed_studies1.collect()

Out[16]: [('Apr', 967),
 ('Aug', 700),
 ('Feb', 934),
 ('Jan', 1131),
 ('Jul', 819),
 ('Jun', 1094),
 ('Mar', 1227),
 ('May', 984),
 ('Oct', 187),
 ('Sep', 528)]

In [0]:
import calendar
month_sorting = {i:e for e, i in enumerate(calendar.month_abbr[1:],1)} 
sorted_completed_studies = Completed_studies1.sortBy(keyfunc=lambda v: month_sorting.get(v[0]))
sorted_completed_studies.collect()

Out[17]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]